In [1]:
import pandas as pd
from operator import itemgetter
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import re
import nltk

from spacy.lang.en import STOP_WORDS
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords

In [2]:
#Long stop words list using 3 differnt python stopword libraries
stops = list(set(stopwords.words('english') + list(set(ENGLISH_STOP_WORDS)) + list(set(STOP_WORDS)) + ["http"]))

In [46]:
#This Block loads the Lexicon and creates a data structure for the emotion-itensity
fileEmotion = "emotion_itensity.txt"
table = pd.read_csv(fileEmotion,  names=["word", "emotion", "intensity"], sep='\t')
lmtzr = WordNetLemmatizer()
lemma_word = []
pos = []
for word in table.word:
    pos_temp= get_wordnet_pos(word)
    pos.append(pos_temp)
    lemma_word.append(lmtzr.lemmatize(word, pos_temp))
table['word'] = lemma_word
table['pos'] = pos

In [47]:
table

,word,emotion,intensity,pos
0,outrage,anger,0.964,v
1,brutality,anger,0.959,n
2,hatred,anger,0.953,v
3,hateful,anger,0.940,n
4,terrorize,anger,0.939,v
...,...,...,...,...
9923,fugitive,trust,0.141,a
9924,divorce,trust,0.133,n
9925,mistake,trust,0.133,n
9926,bait,trust,0.133,n


In [59]:
temp = table.groupby(['word', 'pos', 'emotion'])['intensity'].mean().unstack('emotion')

In [70]:
words = [i[0] for i in temp.index]
pos = [i[1] for i in temp.index]

In [73]:
new = pd.DataFrame()
new['word'] = words
new['pos'] = pos
new['anger'] = list(temp['anger'])
new['anticipation'] = list(temp['anticipation'])
new['disgust'] = list(temp['disgust'])
new['fear'] = list(temp['fear'])
new['joy'] = list(temp['joy'])
new['sadness'] = list(temp['sadness'])
new['suprise'] = list(temp['surprise'])
new['trust'] = list(temp['trust'])
new["sum"] = new.sum(axis=1)
new.fillna(0)

,word,pos,anger,anticipation,disgust,fear,joy,sadness,suprise,trust,sum
0,aaaaaaah,n,0.000,0.00,0.0,0.344,0.000,0.000,0.000,0.000,0.344
1,aaaah,n,0.000,0.00,0.0,0.234,0.000,0.000,0.000,0.000,0.234
2,abacus,n,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.406,0.406
3,abandon,n,0.000,0.00,0.0,0.531,0.000,0.703,0.000,0.000,1.234
4,abandon,v,0.222,0.00,0.0,0.534,0.000,0.828,0.000,0.000,1.584
...,...,...,...,...,...,...,...,...,...,...,...
5780,zany,n,0.000,0.00,0.0,0.000,0.000,0.000,0.555,0.000,0.555
5781,zeal,n,0.000,0.50,0.0,0.000,0.547,0.000,0.484,0.328,1.859
5782,zealous,a,0.000,0.00,0.0,0.000,0.393,0.000,0.000,0.258,0.651
5783,zen,n,0.000,0.00,0.0,0.000,0.515,0.000,0.000,0.000,0.515


In [74]:
new.to_csv('nrc_eil.csv')

In [25]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [26]:
lmtzr = WordNetLemmatizer()
lemma_word = []
pos = []
for word in table.word:
    pos_temp= get_wordnet_pos(word)
    pos.append(pos_temp)
    lemma_word.append(lmtzr.lemmatize(word, pos_temp))

In [22]:
table.pivot(index='word', columns='emotion', values='intensity', fill_value=0)

emotion,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
word,,,,,,,,
aaaaaaah,NaN,NaN,NaN,0.344,NaN,NaN,NaN,NaN
aaaah,NaN,NaN,NaN,0.234,NaN,NaN,NaN,NaN
abacus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.406
abandon,NaN,NaN,NaN,0.531,NaN,0.703,NaN,NaN
abandoned,0.222,NaN,NaN,0.534,NaN,0.828,NaN,NaN
...,...,...,...,...,...,...,...,...
zany,NaN,NaN,NaN,NaN,NaN,NaN,0.555,NaN
zeal,NaN,0.50,NaN,NaN,0.547,NaN,0.484,0.328
zealous,NaN,NaN,NaN,NaN,0.393,NaN,NaN,0.258


KeyError: 'None of [None] are in the columns'

In [ ]:
#function that get the emotion itensity
def getEmotionItensity(word,emotion):
    key = word + "#" + emotion
    try:
        return emotion_dic[key]
    except:
        return 0.0

In [ ]:
#Check if the word is in the Lexicon
def isWordInEmotionFile(word):
    result = [(key) for key in emotion_dic.keys() if key.startswith(word + "#")]
    if len(result) == 0:
        return False
    else:
        return True

In [ ]:
#Stopping checker 
def isStopWord(word):
    if word in stops:
        return True
    else:
        return False

In [ ]:
#Assign the emotion itensity to the dictionary
def calculateEmotion(emotions, word):
    emotions["Anger"] += getEmotionItensity(word, "anger")
    emotions["Anticipation"] += getEmotionItensity(word, "anticipation")
    emotions["Disgust"] += getEmotionItensity(word, "disgust")
    emotions["Fear"] += getEmotionItensity(word, "fear")
    emotions["Joy"] += getEmotionItensity(word, "joy")
    emotions["Sadness"] += getEmotionItensity(word, "sadness")
    emotions["Surprise"] += getEmotionItensity(word, "surprise")
    emotions["Trust"] += getEmotionItensity(word, "trust")

In [ ]:
#get the emotion vector of a given text
def getEmotionVector(text):
    #create the initial emotions
    emotions = {"Anger": 0.0,
                "Anticipation": 0.0,
                "Disgust": 0.0,
                "Fear": 0.0,
                "Joy": 0.0,
                "Sadness": 0.0,
                "Surprise": 0.0,
                "Trust": 0.0,
                "Objective": 0.0}
    #parse the description
    str = re.sub("[^a-zA-Z]+", " ", text)
    pat = re.compile(r'[^a-zA-Z ]+')
    str = re.sub(pat, '', str).lower()
    #split string
    splits = str.split()

    #iterate over words array
    for split in splits:
        if not isStopWord(split):
            #first check if the word is in its natural form
            if isWordInEmotionFile(split): 
                calculateEmotion(emotions, split)
            elif isWordInEmotionFile(lmtzr.lemmatize(split)):
                calculateEmotion(emotions, lmtzr.lemmatize(split))
            elif isWordInEmotionFile(lmtzr.lemmatize(split, 'v')):
                calculateEmotion(emotions, lmtzr.lemmatize(split, 'v'))   
            else:
                emotions["Objective"] += 1
    total = sum(emotions.values())
    for key in sorted(emotions.keys()):
        try:
            emotions[key] = (1.0 / total) * emotions[key]
        except:
            emotions[key] = 0
    return emotions

### Example

In [ ]:
getEmotionVector("Bob is happy.")